In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/upcoming-event.csv')
len(df)

4280

In [6]:
num_upcoming_fights = len(df_upcoming)

In [7]:
df = df_upcoming.append(df)
len(df)

4292

In [8]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [10]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [11]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

12
12
12
4280
4280
4280


In [12]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,label
0,Calvin Kattar,Dan Ige,-286,220,29.850746,245.0,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,10.0,NaN,NaN,NaN,Red,U-DEC,NaN,0.0
1,Tim Elliott,Ryan Benoit,-141,112,80.000000,100.0,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,0.0
2,Jimmie Rivera,Cody Stamann,-141,112,70.921986,112.0,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,10.0,NaN,NaN,neither,U-DEC,NaN,0.0
3,Molly McCann,Taila Santos,-152,120,80.000000,100.0,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,1.0
4,Abdul Razak Alhassan,Mounir Lazzez,-345,260,28.985507,260.0,2020-07-15,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4275,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.0,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,NaN,1.0
4276,John Howard,Daniel Roberts,-210,175,47.619048,175.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punch,0.0
4277,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punches,0.0
4278,Mike Pierce,Julio Paulino,-420,335,23.809524,335.0,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,label
0,Deiveson Figueiredo,Joseph Benavidez,-225,180,44.444444,180.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,Red,NaN,NaN,NaN
1,Jack Hermansson,Kelvin Gastelum,-112,-112,89.285714,89.285714,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,NaN
2,Marc Diakiese,Rafael Fiziev,-167,135,59.880240,135.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
3,Ariane Lipski,Luana Carolina,-125,100,80.000000,100.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
4,Alexandre Pantoja,Askar Askarov,-210,165,47.619048,165.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
5,Roman Dolidze,Khadis Ibragimov,-167,135,59.880240,135.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
6,Grant Dawson,Nad Narimani,-250,195,40.000000,195.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
7,Joe Duffy,Joel Alvarez,-335,250,29.850746,250.000000,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
8,Brett Johns,Montel Jackson,170,-215,170.000000,46.511628,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
9,Malcolm Gordon,Amir Albazi,135,-167,135.000000,59.880240,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN


In [13]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [14]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    
    return(0)

In [15]:
with open('../automated_model_creation/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice:
model_choice = 8

print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'DecisionTreeClassifier(random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=50, n_estimators=15,\n                       n_jobs=-1, random_state=75)", 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB(var_smoothing=1e-10)', 'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)'], ["['B_Weight_lbs', 'B_win_by_Decision_Unanimous', 'R_win_by_Decision_Split', 'R_draw', 'B_avg_TD_landed', 'sig_str_dif', 'B_current_win_streak', 'B_Height_cms', 'R_win_by_TKO_Doctor_Stoppage', 'R_win_by_Submission', 'win_streak_dif', 'R_longest_win_streak', 'R_losses', 'B_longest_win_streak', 'R_total_title_bouts', 'R_current_win_streak', 'ti

In [16]:

get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Deiveson Figueiredo vs. Joseph Benavidez
Deiveson Figueiredo has an EV of 8.33 on odds of -225.  They have 74.99 % chance of winning.

Jack Hermansson vs. Kelvin Gastelum
Jack Hermansson has an EV of 0.88 on odds of -112.  They have 53.3 % chance of winning.

Marc Diakiese vs. Rafael Fiziev
Rafael Fiziev has an EV of 5.35 on odds of 135.  They have 44.83 % chance of winning.

Ariane Lipski vs. Luana Carolina
No bets.  EV1:-1.1458193660076859. EV2: -9.837978482213686
Ariane Lipski has an EV of -1.15 on odds of -125.  They have 54.92 % chance of winning.

Alexandre Pantoja vs. Askar Askarov
Askar Askarov has an EV of 28.78 on odds of 165.  They have 48.6 % chance of winning.

Roman Dolidze vs. Khadis Ibragimov
Khadis Ibragimov has an EV of 8.38 on odds of 135.  They have 46.12 % chance of winning.

Grant Dawson vs. Nad Narimani
Grant Dawson has an EV of 10.02 on odds of -250.  They have 78.59 % chance of winning.

Joe Duffy vs. Joel Alvarez
Joe Duffy has an EV of 7.07 on odds of -335.  T

0